In [ ]:
#My VS Code has issues connecting to the venv.
# I find that first connecting to one of my conda environments
# and then switching to the venv works.
# I run this cell on conda environment first then switch to venv
# and run it there and seems to be all good.
print("Hello")

In [ ]:
# This cell allows hot reloading of modules.
# It is useful for development, so you don't have to restart the kernel.
# Some changes such as adding new files or changing the structure of the code
# may not be picked up and will require a kernel restart.
%load_ext autoreload
%autoreload 2

# Testing LINZ with GeoPandas  
This notebook is for testing a python environment that has GeoPandas installed, using LINZ as the testing portal.  

This notebook expects GeoPandas and not the arcgis module. This will force the default data types to use GeoDataFrames.  

In [ ]:
import os
import json
import time
import geopandas as gpd
from dotenv import load_dotenv, find_dotenv
import logging

# Set up logging include logger name
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

from kapipy.gis import GIS
from kapipy.gis import has_geopandas, has_arcgis, has_arcpy

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
api_key = os.getenv('LINZ_API_KEY')

output_folder = r"c:\temp\data"

#layers
rail_station_layer_id = "50318" #rail station 175 points
fences_layer_id = "50268" #NZ Fence Centrelines  
geodetic_marks_layer_id = "50787" # NZ Geodetic Marks 132,966 point features
nz_walking_biking_tracks_layer_id = "52100" #NZ Walking and Biking Tracks 29,187 polyline features

#tables
nz_building_age_table_id = "105617" #NZ Building Age 18 records, no changeset
suburb_locality_table_id = "113761" #NZ Suburb Locality 3190 records

print(f'{has_arcgis=}')
print(f'{has_arcpy=}')
print(f'{has_geopandas=}')

assert(has_geopandas == True)
assert(has_arcgis == False)


## Server object  
Get a reference to the LINZ server  

In [ ]:
linz = GIS(name="linz", api_key=api_key)
print(linz)

## Layer item  
Run some tests on a layer item  

In [ ]:
itm = linz.content.get(geodetic_marks_layer_id)
print(itm)

In [ ]:
print(f'{itm.supports_changesets=}')
print(f'{itm.data.fields=}')
print(f'{itm.data.crs=}')
print(f'{itm.data.geometry_type=}')

In [ ]:
itm.data.fields[0]

In [ ]:
waikato_polygon = gpd.read_file('../examples/waikato.json')
print(waikato_polygon)

# read shapefile into a GeoDataFrame
matamata_gdf = gpd.read_file("../examples/matamata_piako.shp")
matamata_gdf.head()

In [ ]:
#Query using WFS
res = itm.query(count=100, wkid = 2193, bbox=matamata_gdf)
logging.info(f"Total records returned {itm.title}: {res.shape[0]}")
res.head()

In [ ]:
# Get a changeset from the layer using WFS
cs_data = itm.changeset(from_time="2024-01-01T00:00:00Z", wkid=2193 )
logging.info(f"Total records returned {itm.title}: {cs_data.shape[0]}")
cs_data.head()

In [ ]:
job = itm.export("geodatabase", wkid=2193, extent=waikato_polygon)

In [ ]:
print(job.status)

In [ ]:
dl = job.download(folder=output_folder)

In [ ]:
print(f'{dl.folder=}')
print(f'{dl.filename=}')
print(f'{dl.file_path=}')
print(f'{dl.file_size_bytes=}')
print(f'{dl.download_url=}')
print(f'{dl.final_url=}')
print(f'{dl.job_id=}')
print(f'{dl.completed_at=}')
print(f'{dl.checksum=}')

## Table item  
Run some tests on a table item

In [ ]:
table_itm = linz.content.get(suburb_locality_table_id)
print(table_itm)

In [ ]:
print(f'{table_itm.supports_changesets=}')
print(f'{table_itm.data.fields=}')

In [ ]:
#Query using WFS to get DataFrame
data = table_itm.query(count=100)
logging.info(f"Total records returned {table_itm.title}: {data.shape[0]}")
data.head()

In [ ]:
#Query using WFS to get raw JSON
data = table_itm.query_json(count=10)
logging.info(f"Total records returned {table_itm.title}: {len(data['features'])}")
print(data)

In [ ]:
# Get a changeset from the layer using WFS
cs_data = table_itm.get_changeset(from_time="2024-01-01T00:00:00Z", count=100)
logging.info(f"Total records returned {table_itm.title}: {cs_data.shape[0]}")
cs_data.head()

In [ ]:
# Get a changeset from the layer using WFS
cs_data = table_itm.get_changeset_json(from_time="2024-01-01T00:00:00Z", count=10)
logging.info(f"Total records returned {table_itm.title}: {len(cs_data['features'])}")
print(cs_data)

In [ ]:
job = table_itm.export("geodatabase")

In [ ]:
print(job)

In [ ]:
job.download(folder=output_folder)

## Test multidownload samples  
Test downloading multiple jobs using the ContentManager download method.    

In [ ]:
itm1 = linz.content.get(geodetic_marks_layer_id)
itm2 = linz.content.get(rail_station_layer_id)
print(itm1)
print(itm2)

In [ ]:
job1 = itm1.export("geodatabase", wkid=2193, extent=matamata_gdf,)
job2 = itm2.export("geodatabase", wkid=2193, extent=matamata_gdf,)

output_folder = r"c:\temp\data\multiple"
results = linz.content.download(jobs=[job1, job2], folder=output_folder)

In [ ]:
for result in results:
    print(result.download_file_path)
